# 上傳檔案 - NTP clock

### 需先安裝 ampy ( Adafruit MicroPython Tool )
#### pip install adafruit-ampy
https://github.com/adafruit/ampy  

In [1]:
import os
import shutil

### 設定COM port (set current COM port)

In [2]:
com_port = 'COM7'
# com_port = 'COM8'
# com_port = 'COM12'
# com_port = 'COM13'
com_port = 'COM15'
# com_port = 'COM16'
# com_port = '/dev/ttyUSB0'

### Utility functions

In [3]:
def clear_local_folder(folder):
    print('\n[Clearing folder {}]'.format(folder))
    for file in os.listdir(folder):
        os.remove(os.path.join(folder, file))
        
        
def copy_one_file_to_local_folder(folder, file, destination_folder):
    print('Copying {} to {}'.format(file, destination_folder))
    shutil.copy(os.path.join(folder, file), destination_folder)
    
    
def copy_all_files_to_local_folder(folders, destination_folder):
    print('\n[Copying all files to upload folder {}]'.format(destination_folder))
    clear_local_folder(destination_folder)    
    
    for folder in folders: 
        for file in os.listdir(folder):
            if (file.endswith('.py') or file.endswith('.mpy')) and not file.startswith('_'):
                copy_one_file_to_local_folder(folder, file, destination_folder)

In [4]:
def list_files_in_device(com_port):
    files = !ampy --port {com_port} ls
    return sorted(files)


def cat_file_from_device(com_port, file):
    !ampy --port {com_port} get {file}
    

def delete_file_in_device(com_port, file):
    print('Deleting {}'.format(file))
    !ampy --port {com_port} rm {file}
    

def delete_files_in_device(com_port):
    print('\n[Deleting all files in device {}]'.format(com_port))
    for file in list_files_in_device(com_port):
        delete_file_in_device(com_port, file)
        
#     try:
#         !ampy --port {com_port} rmdir {'/'}
#     except Exception as e:
#         print(e)        
        
        
def copy_one_file_to_device(com_port, folder, file, mpy_only = False):
    if mpy_only:
        if file.endswith('.mpy'):
            print('Copying {}'.format(file))
            !ampy --port {com_port} put {os.path.join(folder, file)}  
    elif file.endswith('.py'):
        print('Copying {}'.format(file))
        !ampy --port {com_port} put {os.path.join(folder, file)}   
        

def delet_main_in_device(com_port, main_file_names):
    print('Deleting {}'.format(main_file_names))
    files = list_files_in_device(com_port)

    for file in main_file_names:
        if file in files:
            delete_file_in_device(com_port, file)
            
            
def delete_main_and_files_in_device(com_port, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):
    print('\n[Copying all files to device {}]'.format(com_port))
    delet_main_in_device(com_port, main_file_names)
    if mpy_only: delete_first = True
    if delete_first: delete_files_in_device(com_port)
        

def copy_all_files_to_device(com_port, folder, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):    
    for file in os.listdir(folder):
        if (file.endswith('.py') or file.endswith('.mpy')) and not file.startswith('_') and not file in main_file_names:
            copy_one_file_to_device(com_port, folder, file, mpy_only)                    
    
    for file in main_file_names:
        if os.path.isfile(os.path.join(folder, file)):
            copy_one_file_to_device(com_port, folder, file, mpy_only)  
            
            
def do_all_to_device(com_port, folder, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):
    delete_main_and_files_in_device(com_port, main_file_names, delete_first, mpy_only)
    copy_all_files_to_device(com_port, folder, main_file_names, delete_first, mpy_only)

### Copy *.py to device (all needed fils will be put in the same folder)

In [5]:
folders = [os.path.sep.join(['..', '..', 'codes', 'clock']),
           os.path.sep.join(['..', '..', 'codes', 'display']),
           os.path.sep.join(['..', '..', 'codes', 'micropython']),
           os.path.sep.join(['..', '..', 'codes', 'node']),]
upload_folder = os.path.sep.join(['upload', 'py'])

copy_all_files_to_local_folder(folders, upload_folder)


[Copying all files to upload folder upload\py]

[Clearing folder upload\py]
Copying ntp_client.py to upload\py
Copying ntp_clock.py to upload\py
Copying display_ssd1306_i2c.py to upload\py
Copying matrix_codes.py to upload\py
Copying max_matrices.py to upload\py
Copying max_matrix.py to upload\py
Copying qs30_1.py to upload\py
Copying shift_register.py to upload\py
Copying tm1629.py to upload\py
Copying wf8266kd.py to upload\py
Copying wf8266kd_commands.py to upload\py
Copying wf8266t.py to upload\py
Copying wf8266t_basic_functions.py to upload\py
Copying ws2812_clock_face.py to upload\py
Copying ws2812_clock_patterns.py to upload\py
Copying boot.py to upload\py
Copying hardware.py to upload\py
Copying led.py to upload\py
Copying main.py to upload\py
Copying node.py to upload\py
Copying worker_upython_oled_clock.py to upload\py


In [6]:
upload_folder_py = os.path.sep.join(['upload', 'py'])

do_all_to_device(com_port, upload_folder, delete_first = True)

print('\n[All done!]')


[Copying all files to device COM15]
Deleting ['main.py', 'main.mpy']
Deleting main.py

[Deleting all files in device COM15]
Deleting asynch_result.py
Deleting boot.py
Deleting commander.py
Deleting config.py
Deleting data_transceiver.py
Deleting display.py
Deleting display_keypad_tm1629.py
Deleting display_ssd1306_i2c.py
Deleting display_tm1629.py
Deleting display_tm1629_wf8266kd.py
Deleting display_tm1629_wf8266r_kd.py
Deleting display_tm1629_wf8266t.py
Deleting hardware.py
Deleting keypad.py
Deleting keypad_tm1629.py
Deleting led.py
Deleting mqtt_client_paho.py
Deleting mqtt_client_umqtt.py
Deleting node.py
Deleting ntp_client.py
Deleting ntp_clock.py
Deleting phonebook.py
Deleting queue_manager.py
Deleting robust.py
Deleting shift_register.py
Deleting simple.py
Deleting socket_client.py
Deleting tm1629.py
Deleting u_python.py
Deleting u_python_profiler.py
Deleting watchdog.py
Deleting webrepl_cfg.py
Deleting wf8266kd.py
Deleting wf8266kd_commands.py
Deleting wf8266t.py
Deleting wor

### Copy *.mpy to device

In [ ]:
upload_folder_py = os.path.sep.join(['upload', 'py'])
upload_folder_mpy = os.path.sep.join(['upload', 'mpy'])

do_all_to_device(com_port, upload_folder_mpy, mpy_only = True)

copy_one_file_to_device(com_port, upload_folder_py, 'boot.py')
copy_one_file_to_device(com_port, upload_folder_py, 'main.py')

print('\n[All done!]')

### 單一檔案上傳 (single file upload, in case needed)

In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'node']), 'node.py') 

In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'node']), 'worker_upython_oled_clock.py') 

In [6]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'ntp_clock.py') 

Copying ntp_clock.py


In [117]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'main.py') 

Copying main.py


In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'tm1629.py') 

In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'wf8266t.py') 

In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'wf8266kd.py') 

In [24]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'max_matrices.py') 

Copying max_matrices.py


In [12]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'shift_register.py') 

Copying shift_register.py


In [118]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'qs30_1.py') 

Copying qs30_1.py


### 列出檔案 (list files)

In [ ]:
# list_files_in_device(com_port)

### 檢查檔案內容 (check file content)

In [ ]:
# cat_file_from_device(com_port, 'ntp_clock.py')

### 連網測試 (network config and test)

In [ ]:
# 連上網路
# import network; nic=network.WLAN(network.STA_IF); nic.active(True); nic.connect('SSID','password');nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF); nic.active(True); nic.connect('Kingnet-70M-$370', '');nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF); nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF);nic.ifconfig();nic.config('mac');nic.ifconfig((['mac',])

In [ ]:
# 發出 http request
# import socket;addr=socket.getaddrinfo('micropython.org',80)[0][-1]
# s = socket.socket();s.connect(addr);s.send(b'GET / HTTP/1.1\r\nHost: micropython.org\r\n\r\n');data = s.recv(1000);s.close()

In [ ]:
# Delete all files
# import u_python;u_python.del_all_files();import os;os.listdir()


### Run Own Broker container on Raspberry Pi
copy folder 'codes' to Raspberry Pi under folder '/data/elastic_network_of_things_with_micropython',  
so Raspberry Pi has folder '/data/elastic_network_of_things_with_micropython/codes'  
then run the command below on Raspberry Pi.

```
docker run -it -p 9662:9662 --name=Broker --hostname=Broker --volume=/data/elastic_network_of_things_with_micropython:/project wei1234c/python_armv7 /bin/sh -c "cd /project/codes/broker && python3 broker.py" 
```

### Run MQTT Broker container on Raspberry Pi  

https://store.docker.com/community/images/pascaldevink/rpi-mosquitto

place your mosquitto.conf in /srv/mqtt/config/  
NOTE: You have to change the permissions of the directories  
to allow the user to read/write to data and log and read from  

config directory
For TESTING purposes you can use chmod -R 777 /srv/mqtt/*  
Better use "-u" with a valid user id on your docker host  

on Raspberry Pi  
copy mqtt/config mqtt/log mqtt/data on to RPi under /data/elastic_network_of_things_with_mqtt_and_micropython/mqtt  

grand permission for user  
sudo chmod -R 777 /data/elastic_network_of_things_with_mqtt_and_micropython/mqtt/*  
```
docker run -it --rm -p 1883:1883 -p 9001:9001 --name=Mosquitto --hostname=Mosquitto \  
--volume=/data/elastic_network_of_things_with_mqtt_and_micropython/mqtt/config:/mqtt/config:ro \
--volume=/data/elastic_network_of_things_with_mqtt_and_micropython/mqtt/data:/mqtt/data \
--volume=/data/elastic_network_of_things_with_mqtt_and_micropython/mqtt/log:/mqtt/log \
pascaldevink/rpi-mosquitto /usr/sbin/mosquitto -v -c /mqtt/config/mosquitto.conf
```